In [25]:
import numpy as np
from algo_DSPYS import signal_quality_assessment_v3, calc_hr, signal_quality_assessment_v3_with_timestamp
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import hilbert, savgol_filter

from utils import *
from statsmodels.tsa.stattools import acf
import os


In [ ]:
!pip install tsfel

In [ ]:
!pip install statsmodels

In [ ]:
!pip install influxdb

In [ ]:
train_data = np.load('Cage_cat_dec24_dec31.npy')

In [ ]:
train_data.shape

In [ ]:
X_train = train_data[:, :1000]
X_train.shape

In [ ]:
X_train_2 = X_train[0,:].reshape(1,-1)
X_train_2.shape
# lab = labels[0,:].reshape(-1,1)
# times = train_data[0,-1].reshape(-1,1)

In [ ]:
timestamp_list = list()
for i in range(train_data.shape[0]):
    print(train_data[i,-1])
    timestamp_list.append(train_data[i,-1])

In [ ]:
str(int(train_data[i,-1]))

In [ ]:
s = X_train[i,:]
s[-1:].shape

In [ ]:
ids = []

In [ ]:
for i in range(10):
    x = X_train[i,:]
    res = signal_quality_assessment_v3(x=x, Fs=100, n_lag=len(x) // 2,
                                       low = 0.7, high = 35,
                                       denoised_method = 'bandpass',
                                       show=True)
    if res[0]:
        X_train_2 = np.concatenate((X_train_2,X_train[i,:].reshape(1,-1)), axis =0)
        # lab = np.concatenate((lab,labels[i,:].reshape(-1,1)),axis=0)
        # times = np.concatenate((times,train_data[i,-1].reshape(-1,1)),axis=0)
        ids.append(i)

In [ ]:
for i in range(1):
    x = X_train[i,:]
    res = signal_quality_assessment_v3_with_timestamp(x=x, Fs=100, n_lag=len(x) // 2,
                                       low = 0.7, high = 35,
                                       denoised_method = 'bandpass',
                                       show=False)
    if res[0]:
        X_train_2 = np.concatenate((X_train_2,X_train[i,:].reshape(1,-1)), axis =0)
        # lab = np.concatenate((lab,labels[i,:].reshape(-1,1)),axis=0)
        # times = np.concatenate((times,train_data[i,-1].reshape(-1,1)),axis=0)
        ids.append(i)

In [ ]:
res

In [ ]:
for i in range(X_train.shape[0]):
    x = X_train[i,:]
    res = signal_quality_assessment_v3(x=x, Fs=100, n_lag=len(x) // 2, 
                                       low = 0.7, high = 35, 
                                       denoised_method = 'bandpass',
                                       show=False)
    if res[0]:
        X_train_2 = np.concatenate((X_train_2,X_train[i,:].reshape(1,-1)), axis =0)
        # lab = np.concatenate((lab,labels[i,:].reshape(-1,1)),axis=0)
        # times = np.concatenate((times,train_data[i,-1].reshape(-1,1)),axis=0)
        ids.append(i)

In [ ]:
X_train_2 = X_train_2[1:,:]
# lab = lab[1:,:]
# times = times[1:,:]

In [ ]:
X_train_2.shape

In [ ]:
X_train = np.copy(X_train_2)

In [ ]:
a = X_train[15,:]

In [ ]:
plt.figure(figsize=(10,2))
plt.plot(a[:500])

In [ ]:
j = 1558

In [ ]:
j = 0

In [ ]:
a = X_train[j,:]
# i+=1
res = signal_quality_assessment_v3(x=a, Fs=100, n_lag=1000 // 2, 
                                        low = 0.7, high = 35, 
                                        denoised_method = 'bandpass',
                                        show=True)
j+=1

In [ ]:
res

In [ ]:
Fs = 100

In [ ]:
# index = 0
# window_size = int(Fs)
# z= hilbert(a) 
# # z = x 
# envelope = np.abs(z)

# sg_win_len = round(0.27 * Fs)
# if sg_win_len%2 == 0:
#     sg_win_len -= 1
# smoothed_envelope = savgol_filter(envelope, sg_win_len, 3, mode='nearest')
# smoothed_envelope = (smoothed_envelope - np.mean(smoothed_envelope))/np.std(smoothed_envelope)

# sg_win_len = round(2.01 * Fs)
# if sg_win_len%2 == 0:
#     sg_win_len -= 1
# trend = savgol_filter(smoothed_envelope, sg_win_len, 6, mode='nearest')
# # smoothed_envelope = smoothed_envelope - trend
# smoothed_envelope = (smoothed_envelope - np.mean(smoothed_envelope))/np.std(smoothed_envelope)
# acf_x = acf(smoothed_envelope, nlags=500)
# acf_x = acf_x / acf_x[0]

In [ ]:
# plt.figure(figsize=(18,2))
# plt.plot(acf_x)

In [ ]:
# peak_ids, _ = signal.find_peaks(acf_x)

In [ ]:
# peak_ids

In [ ]:
acf_x = res[-1]
peak_ids, _ = signal.find_peaks(acf_x) #height = np.mean(acf_x)
time_diff = peak_ids[1:] - peak_ids[:-1]
time_diff

In [ ]:
peak_ids

In [ ]:
# peak_ids, _ = signal.find_peaks(acf_x, height = np.mean(acf_x))

In [ ]:
fs =100

In [ ]:
cadidates = []
for peak_id in peak_ids:
            if (peak_id > int(0.25 * fs) and peak_id < int(0.8 * fs)):
                cadidates.append(peak_id)
        
            if len(cadidates) == 0:
                median_hr = np.median(time_diff)
            else:
                # median_hr = cadidates[np.argmax(acf_x[cadidates])] #+ int(0.5 * Fs)   
                median_hr = cadidates[0] 

In [ ]:
frequency = 1/(median_hr/fs)

In [ ]:
frequency

In [ ]:
round(frequency * 60) 

In [ ]:
Y_train[5]

In [ ]:
hr = []
label_ac = []

In [ ]:
# print('===============================')
# print('bandpass filter......')
# cnt = 0
# for i in range(X_train.shape[0]):
#     X_train[i, :] = filter_data(X_train[i, :], Fs, 0.7, 35, 5)

In [ ]:
X_train_3 = X_train[0,:].reshape(1,-1)

In [ ]:
for i in range(X_train.shape[0]):
    hr_1, _ = calc_hr(X_train[i,:])
    if hr_1!=-1:
        hr.append(hr_1)
        label_ac.append(Y_train[i,:])
        X_train_3 = np.concatenate((X_train_3,X_train[i,:].reshape(1,-1)),axis=0)

In [ ]:
X_train_3 = X_train_3[1:,:]

In [ ]:
# hr[80:90]

In [ ]:
plt.plot(hr)
plt.ylabel('Heart Rate')
plt.xlabel('Sample')
plt.title('Predictions')

In [ ]:
hr = np.array(hr).reshape(-1,1)
lab_ac = np.array(label_ac).reshape(-1,1)

In [ ]:
# v = int(lab.shape[0])

In [ ]:
hr.shape

In [ ]:
# hr = hr[:v,:]

In [ ]:
plot_2vectors(lab_ac, hr, name='Prediction Results')

In [ ]:
ids_2 = []

In [ ]:
my_hr = []
my_lab = []

In [ ]:
for i in range(hr.shape[0]):
    diff = hr[i,0] - lab[i]
    if 20 < abs(diff) < 25:
        ids_2.append(i)
    if abs(diff)<25:
        my_hr.append(hr[i])
        my_lab.append(lab[i])


In [ ]:
my_hr = np.array(my_hr)
my_lab = np.array(my_lab)

In [ ]:
plot_2vectors(my_lab, my_hr, name='Prediction Results')

In [ ]:
# ids_2 = []

In [ ]:
# for i in range(my_hr.shape[0]):
#     diff = my_hr[i,0] - my_lab[i]
#     if abs(diff)> 20:
#         ids_2.append(i)

In [ ]:
len(ids_2)

In [ ]:
j = 0

In [ ]:
# print('id:', ids_2[j])
# print('pred:', hr[ids_2[j]])
# print('label', lab[ids_2[j]])
# # j+=1

In [ ]:
times[138,0]

In [ ]:
a = X_train_3[2,:]
res = signal_quality_assessment_v3(x=a, Fs=100, n_lag=1000 // 2, 
                                        low = 0.7, high = 35, 
                                        denoised_method = 'bandpass',
                                        show=True)

In [ ]:
acf_x = res[-1]
peak_ids, _ = signal.find_peaks(acf_x, height = np.mean(acf_x)) #height = np.mean(acf_x)

In [ ]:
peak_ids

In [ ]:
time_diff = peak_ids[1:] - peak_ids[:-1]
np.median(time_diff)

In [ ]:
hrr = 1/(peak_ids[0]/fs)*60
hrr

In [ ]:
my_lab[1]